## Import necessary libraries

In [1]:
import numpy as np

## Data

In [2]:
training = r"VISUAL DEFECTS\Ceramic Tile Surface Defects.v1-version-1.multiclass\train"
testing = r"VISUAL DEFECTS\Ceramic Tile Surface Defects.v1-version-1.multiclass\test"
validation = r"VISUAL DEFECTS\Ceramic Tile Surface Defects.v1-version-1.multiclass\valid"

In [4]:
import pandas as pd
from pathlib import Path
import torch

train_csv_path = Path("train\_classes.csv")
train_data = pd.read_csv(train_csv_path)

test_csv_path = Path("test\_classes.csv")
test_data = pd.read_csv(test_csv_path)

valid_csv_path = Path("valid\_classes.csv")
valid_data = pd.read_csv(valid_csv_path)

train_data

filename   crack   scratch  \
0     captured_image_1716845478-1050267_png.rf.ff04b...       0         0   
1     IMG_1276_000012_jpg.rf.ff153c7a0fdd4cf94f8f216...       1         0   
2     IMG_1269_000027_jpg.rf.ff15635b2d93d7abb7b6d08...       1         0   
3     _MG_2746_jpg.rf.ff4b1198a5eeefeb7a53276fc3e835...       0         0   
4     _MG_3154_jpg.rf.fff8d0e3e66fe7fac5039bda39ff26...       1         1   
...                                                 ...     ...       ...   
1672  _MG_2620_jpg.rf.d986e5de6708d9d38804d581fa8986...       0         0   
1673  _MG_3233_jpg.rf.d95b9f3dca3ff93283af9fe9aa6cbf...       0         1   
1674  _MG_3291_jpg.rf.d98127d882c9e24af18cc827778ee8...       0         1   
1675  _MG_2578_jpg.rf.d993600a76166c6d025b95fd5dc7b2...       0         1   
1676  IMG_1268_000034_jpg.rf.d94ca239b5cf2f457d573d9...       1         0   

       stain  
0          1  
1          0  
2          0  
3          1  
4          1  
...      ...  
1672       1  
1673       1  
1674       0  
1675       0  
1676       0  

[1677 rows x 4 columns]

In [5]:
# Associate file paths with their labels
train_data['image_path'] = train_data['filename'].apply(lambda x: str(Path(r"VISUAL DEFECTS\Ceramic Tile Surface Defects.v1-version-1.multiclass\train") / x))
test_data['image_path'] = test_data['filename'].apply(lambda x: str(Path(r"VISUAL DEFECTS\Ceramic Tile Surface Defects.v1-version-1.multiclass\test") / x))
valid_data['image_path'] = valid_data['filename'].apply(lambda x: str(Path(r"VISUAL DEFECTS\Ceramic Tile Surface Defects.v1-version-1.multiclass\valid") / x))

In [6]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=32, shuffle=True)

## Define CNN model

### Inception Model

In [10]:
import torch.nn as nn
from torchvision import models

num_classes = 3

# Load pre-trained model
resnet_model = models.resnet50()
resnet_model.fc = nn.Sequential(
    nn.Linear(resnet_model.fc.in_features, num_classes),
    nn.Sigmoid() # for multi-label classification
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet_model = resnet_model.to(device)

# Loss and optimizer
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = torch.optim.Adam(resnet_model.parameters(), lr=1e-3)

In [ ]:
from tqdm import tqdm

def train_model(model, dataloader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in tqdm(dataloader):
            # Move to GPU if available
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(dataloader)}")

train_model(resnet_model, train_loader, criterion=criterion, optimizer=optimizer, epochs=15)